In [1]:
import json
import pandas as pd
import numpy as np

# Read Json

In [2]:
half = "H1"
### CHOOSE ONE
### "H1", "H2", None

In [3]:
with open("../matches/matches_England.json", "r") as file:
    match_data_2 = json.load(file)

df2 = pd.DataFrame(match_data_2)
if half == "H1":
    df2 = df2[df2.gameweek<20]
elif half == "H2":
    df2 = df2[df2.gameweek>=20]

with open("../matches/matches_France.json", "r") as file:
    match_data_3 = json.load(file)

df3 = pd.DataFrame(match_data_3)
if half == "H1":
    df3 = df3[df3.gameweek<20]
elif half == "H2":
    df3 = df3[df3.gameweek>=20]


with open("../matches/matches_Germany.json", "r") as file:
    match_data_4 = json.load(file)

df4 = pd.DataFrame(match_data_4)
if half == "H1":
    df4 = df4[df4.gameweek<18]
elif half == "H2":
    df4 = df4[df4.gameweek>=18]

with open("../matches/matches_Italy.json", "r") as file:
    match_data_5 = json.load(file)

df5 = pd.DataFrame(match_data_5)
if half == "H1":
    df5 = df5[df5.gameweek<20]
elif half == "H2":
    df5 = df5[df5.gameweek>=20]

with open("../matches/matches_Spain.json", "r") as file:
    match_data_6 = json.load(file)

df6 = pd.DataFrame(match_data_6)
if half == "H1":
    df6 = df6[df6.gameweek<20]
elif half == "H2":
    df6 = df6[df6.gameweek>=20]

all_matches_df = pd.concat([df2, df3, df4, df5, df6], ignore_index=True)
all_matches_list = list(np.unique(all_matches_df.wyId))

In [ ]:
half = "H1"
### CHOOSE ONE
### H1, H2, None

with open("../matches/matches_England.json", "r") as file:
    match_data_2 = json.load(file)

df2 = pd.DataFrame(match_data_2)
if half == "H1":
    df2 = df2[df2.gameweek<20]
elif half == "H2":
    df2 = df2[df2.gameweek>=20]

with open("../matches/matches_France.json", "r") as file:
    match_data_3 = json.load(file)

df3 = pd.DataFrame(match_data_3)
if half == "H1":
    df3 = df3[df3.gameweek<20]
elif half == "H2":
    df3 = df3[df3.gameweek>=20]


with open("../matches/matches_Germany.json", "r") as file:
    match_data_4 = json.load(file)

df4 = pd.DataFrame(match_data_4)
if half == "H1":
    df4 = df4[df4.gameweek<18]
elif half == "H2":
    df4 = df4[df4.gameweek>=18]

with open("../matches/matches_Italy.json", "r") as file:
    match_data_5 = json.load(file)

df5 = pd.DataFrame(match_data_5)
if half == "H1":
    df5 = df5[df5.gameweek<20]
elif half == "H2":
    df5 = df5[df5.gameweek>=20]

with open("../matches/matches_Spain.json", "r") as file:
    match_data_6 = json.load(file)

df6 = pd.DataFrame(match_data_6)
if half == "H1":
    df6 = df6[df6.gameweek<20]
elif half == "H2":
    df6 = df6[df6.gameweek>=20]

all_matches_df = pd.concat([df2, df3, df4, df5, df6], ignore_index=True)

with open("../events/events_England.json", "r") as file:
    event_data_2 = json.load(file)

df2 = pd.DataFrame(event_data_2)
df2 = df2[df2.matchId.isin(all_matches_list)]

with open("../events/events_France.json", "r") as file:
    event_data_3 = json.load(file)

df3 = pd.DataFrame(event_data_3)
df3 = df3[df3.matchId.isin(all_matches_list)]


with open("../events/events_Germany.json", "r") as file:
    event_data_4 = json.load(file)

df4 = pd.DataFrame(event_data_4)
df4 = df4[df4.matchId.isin(all_matches_list)]


with open("../events/events_Italy.json", "r") as file:
    event_data_5 = json.load(file)

df5 = pd.DataFrame(event_data_5)
df5 = df5[df5.matchId.isin(all_matches_list)]


with open("../events/events_Spain.json", "r") as file:
    event_data_6 = json.load(file)

df6 = pd.DataFrame(event_data_6)
df6 = df6[df6.matchId.isin(all_matches_list)]

with open("../events/teams.json", "r") as file:
    teams_data = json.load(file)

teams_df = pd.DataFrame(teams_data)

In [ ]:
all_events_df = pd.concat([df2, df3, df4, df5, df6], ignore_index=True)

In [ ]:
passes_df = all_events_df[all_events_df['eventName'] == 'Pass']
in_possession_df = all_events_df[all_events_df['eventName'].isin(['Shot','Pass'])]

In [ ]:
# Determine the previous team ID in the same match
passes_df['previous_teamId'] = passes_df.groupby('matchId')['teamId'].shift(1)

# Identify transitions where the team ID changes and it's within the same match
transitions = passes_df[(passes_df['teamId'] != passes_df['previous_teamId']) & (passes_df['previous_teamId'].notna())]

# Create a transitions DataFrame with required columns for merging
transitions['endSec'] = transitions['eventSec'] + 10
transition_times = transitions[['id', 'matchId', 'eventSec', 'endSec']]

In [ ]:
# Convert data types to more memory efficient types
transitions['matchId'] = transitions['matchId'].astype('int32')
transitions['eventSec'] = transitions['eventSec'].astype('float32')
transitions['endSec'] = transitions['endSec'].astype('float32')

# Check if converting teamId to a smaller integer type is possible
transitions['teamId'] = pd.to_numeric(transitions['teamId'], downcast='integer')

In [ ]:
passes_df['previous_teamId'] = passes_df.groupby('matchId')['teamId'].shift(1)
transitions = passes_df[(passes_df['teamId'] != passes_df['previous_teamId']) & (passes_df['previous_teamId'].notna())]

# Add a 10-second window for the transition end time
transitions['transition_end_time'] = transitions['eventSec'] + 10

# Prepare the minimal necessary data for merging
transitions_minimal = transitions[['id', 'matchId', 'eventSec', 'transition_end_time']]
in_possession_df_minimal = in_possession_df[['id', 'matchId', 'eventSec', 'teamId', "subEventName", "positions", "eventName", "playerId"]]

In [ ]:
# Define a function to process chunks
def process_chunk(chunk):
    chunk_merged = chunk.merge(transitions_minimal, on='matchId', suffixes=('', '_trans'))
    return chunk_merged[(chunk_merged['eventSec'] >= chunk_merged['eventSec_trans']) & (chunk_merged['eventSec'] <= chunk_merged['transition_end_time'])]

# Process in chunks
chunk_size = 100000
result_frames = []
for start in range(0, in_possession_df_minimal.shape[0], chunk_size):
    chunk = in_possession_df_minimal.iloc[start:start + chunk_size]
    result_frames.append(process_chunk(chunk))

In [ ]:
in_poss_merged_df = pd.concat(result_frames)
in_poss_merged_df = in_poss_merged_df.drop_duplicates(subset='id', keep='first')

In [ ]:
in_poss_merged_df

# Feature engineering - Pass DF - Pass Motifs

In [ ]:
passes_df = in_poss_merged_df

# Concatenating teamId and matchId to create a unique identifier for each team in each match
passes_df['uniqueTeamId'] = passes_df['teamId'].astype(str) + "---" + passes_df['matchId'].astype(str)

In [ ]:
# Function to classify pass motifs
def classify_pass_motif(passes):
    players = passes['playerId'].values
    if len(set(players)) == 2:  # ABAB
        return 'ABAB'
    elif len(set(players)) == 3:
        if players[0] == players[2] or players[1] == players[3]:  # ABCA or ABCB
            return 'ABCA' if players[0] == players[2] else 'ABCB'
        else:
            return 'ABCD'  # ABCD for distinct players or other scenarios
    return 'ABCD'

# Apply the classification within a rolling window
def apply_motif_classification(df):
    # Ensure the DataFrame is sorted by event time
    df = df.sort_values(by='eventSec')
    motifs = []
    # Using a rolling window of 4 events
    for i in range(len(df) - 3):
        window = df.iloc[i:i+4]
        motif = classify_pass_motif(window)
        motifs.append((window.iloc[-1]['uniqueTeamId'], motif))
    return pd.DataFrame(motifs, columns=['uniqueTeamId', 'Motif'])

In [ ]:
motif_classification_results = passes_df.groupby('uniqueTeamId').apply(apply_motif_classification)

In [ ]:
motif_classification_results.reset_index(drop=True, inplace=True)

In [ ]:
motif_counts = motif_classification_results.groupby(['uniqueTeamId', 'Motif']).size().unstack(fill_value=0)

In [ ]:
motif_ratios = motif_counts.div(motif_counts.sum(axis=1), axis=0)
# df to be used in clustering

In [ ]:
motif_ratios

In [ ]:
motif_stats = motif_ratios.agg(['mean', 'std']).T

In [ ]:
motif_stats

In [ ]:
motif_ratios.to_csv('transition_motif_ratios_df.csv', index=True)

# Feature engineering - Shot DF

In [ ]:
shot_df = in_poss_merged_df[in_poss_merged_df['eventName'] == 'Shot']
shot_df['uniqueTeamId'] = shot_df['teamId'].astype(str) + "---" + shot_df['matchId'].astype(str)

In [ ]:
import math

def calculate_shot_distance(row):
    if row['positions'] is not None and len(row['positions']) > 0:
        loc = row['positions'][0]
        # Normalize locations
        x = loc['x'] * (120 / 100.0)  # Normalizing the x coordinate
        y = loc['y'] * (80 / 100.0)   # Normalizing the y coordinate
        # Calculate Euclidean distance from (0,0)
        distance = math.sqrt((120-x)**2 + (80-y)**2)
        return distance
    return None

# Apply the function to calculate shot distance and add it to the DataFrame
shot_df['shot_distance'] = shot_df.apply(calculate_shot_distance, axis=1)

In [ ]:
def classify_shot_by_distance(distance):
    if distance <= 38:
        return "Near"
    elif distance <= 52:
        return "Middle Distance"
    else:
        return "Far"

# Assuming shot_df has a 'shot_distance' column calculated as before
shot_df['shot_class'] = shot_df['shot_distance'].apply(classify_shot_by_distance)

# Grouping shots by their class and counting
shot_class_counts = shot_df.groupby('uniqueTeamId')['shot_class'].value_counts().unstack(fill_value=0)
shot_counts = shot_df.groupby('uniqueTeamId')['subEventName'].value_counts().unstack(fill_value=0)

In [ ]:
shot_df_final = pd.concat([shot_counts, shot_class_counts], axis=1)

In [ ]:
shot_df_final

In [ ]:
shot_df_final.rename(columns = {'Shot':'Total Shots'}, inplace = True) 

shot_df_final["Far"] = shot_df_final["Far"]/shot_df_final["Total Shots"]
shot_df_final["Middle Distance"] = shot_df_final["Middle Distance"]/shot_df_final["Total Shots"]
shot_df_final["Near"] = shot_df_final["Near"]/shot_df_final["Total Shots"]


shot_df_final.rename(columns = {'Near':'Near Shots Ratio', 'Middle Distance':'Middle Distance Shots Ratio', 'Far':'Far Shots Ratio',}, inplace = True) 

In [ ]:
shot_df_final

# Feature engineering - Pass DF - Pass Types

In [ ]:
def determine_zone(position):
    if 'x' in position and 'y' in position:
        x, y = position['x'], position['y']
        zone = int(x / 33.333) + 1
        if zone == 4:
            zone = 3
        return zone
    else:
        return None
    
# Apply the function to determine zones for each event
passes_df['Zone'] = passes_df['positions'].apply(lambda pos: determine_zone(pos[0]) if len(pos) > 0 else None)

In [ ]:
passes_df

In [ ]:
import math

def calculate_pass_details_from_single_column(row):
    if row['positions'] is not None and len(row['positions']) == 2:
        # Extract and normalize start and end locations
        start_loc = row['positions'][0]
        end_loc = row['positions'][1]
        
        start_x = start_loc['x'] * (120 / 100.0)
        start_y = start_loc['y'] * (80 / 100.0)
        end_x = end_loc['x'] * (120 / 100.0)
        end_y = end_loc['y'] * (80 / 100.0)
        
        # Calculate pass length and angle
        delta_x = end_x - start_x
        delta_y = end_y - start_y
        pass_length = math.sqrt(delta_x**2 + delta_y**2)
        pass_angle = math.atan2(delta_y, delta_x)
        
        return pd.Series([pass_length, pass_angle])
    else:
        return pd.Series([None, None])

In [ ]:
passes_df[['calculated_pass_length', 'calculated_pass_angle']] = passes_df.apply(calculate_pass_details_from_single_column, axis=1)


In [ ]:
def determine_pass_direction(angle):
    # Forward Pass: -π/4 < angle < π/4
    if -np.pi/4 < angle < np.pi/4:
        return "Forward Pass"
    # Backward Pass: 3π/4 < angle <= π or -π <= angle < -3π/4
    elif 3*np.pi/4 < angle <= np.pi or -np.pi <= angle < -3*np.pi/4:
        return "Backward Pass"
    # Side Pass: Otherwise
    else:
        return "Side Pass"

In [ ]:
passes_df['pass_direction'] = passes_df['calculated_pass_angle'].apply(determine_pass_direction)

In [ ]:
passes_df

In [ ]:
np.unique(passes_df.subEventName)

In [ ]:
passes_df

In [ ]:
#passes_count = passes_df.groupby('teamId')['subEventName'].value_counts().unstack(fill_value=0)

In [ ]:
"""
passes_df['pass_direction'] = passes_df['calculated_pass_angle'].apply(determine_pass_direction)
passes_df['dir_zone'] = passes_df['pass_direction'].astype(str) + '_Zone_' + passes_df['Zone'].astype(str)

# Analyzing pass length
q1, q2, q3 = passes_df['calculated_pass_length'].quantile([0.33, 0.66, 1.0])
passes_df['pass_length_type'] = pd.cut(passes_df['calculated_pass_length'], bins=[0, q1, q2, q3], labels=['Short', 'Middle', 'Long'], include_lowest=True)

# Grouping, Counting, and Pivoting
grouped = passes_df.groupby(['uniqueTeamId', 'dir_zone']).size()
pivot_table = grouped.unstack(fill_value=0)
pass_length_stats = passes_df.groupby(['uniqueTeamId', 'pass_length_type']).size().unstack(fill_value=0)
average_pass_length = passes_df.groupby('uniqueTeamId')['calculated_pass_length'].mean()

# Final DataFrame
final_df = pd.concat([pivot_table, pass_length_stats, average_pass_length.rename('average_pass_length')], axis=1)

final_df.head()  # Display the top rows of the DataFrame"""


In [ ]:
# Step 1: Combine 'pass_direction' and 'Zone' into a single column
passes_df['dir_zone'] = passes_df['pass_direction'].astype(str) + '_Zone_' + passes_df['Zone'].astype(str)

# Step 2: Group by 'uniqueTeamId' and the new 'dir_zone' column, then count
direction_zone_count = passes_df.groupby(['uniqueTeamId', 'dir_zone']).size()

q1, q2, q3 = passes_df['calculated_pass_length'].quantile([0.33, 0.66, 1.0])
passes_df['pass_length_type'] = pd.cut(passes_df['calculated_pass_length'], bins=[0, q1, q2, q3], labels=['Short', 'Middle', 'Long'], include_lowest=True)
pass_length_stats = passes_df.groupby(['uniqueTeamId', 'pass_length_type']).size().unstack(fill_value=0)
average_pass_length = passes_df.groupby('uniqueTeamId')['calculated_pass_length'].mean()


# Step 3: Pivot the results to have 'uniqueTeamId' as rows and each unique 'dir_zone' combination as columns
direction_zone_count_pivot = direction_zone_count.unstack(fill_value=0)

direction_count = passes_df.groupby('uniqueTeamId')['pass_direction'].value_counts().unstack(fill_value=0)

pass_direction_df = pd.concat([direction_zone_count_pivot, direction_count, pass_length_stats, average_pass_length.rename('average_pass_length')], axis=1)

In [ ]:
pass_direction_df

## Pass Types

In [ ]:
high_pass_types = ['High pass', 'Cross', 'Launch']
# Define low pass types based on exclusion
all_pass_types = passes_df['subEventName'].unique()
low_pass_types = [pt for pt in all_pass_types if pt not in high_pass_types and pt != 'Smart pass']

# Group by 'uniqueTeamId' and count passes
pass_counts = passes_df.groupby(['uniqueTeamId', 'subEventName']).size().unstack(fill_value=0)
pass_counts['Total_Passes'] = pass_counts.sum(axis=1)

# Calculate high pass and low pass totals
pass_counts['High_pass_Total'] = pass_counts[high_pass_types].sum(axis=1)
pass_counts['Low_pass_Total'] = pass_counts[low_pass_types].sum(axis=1)

# Calculate overall high pass and low pass ratios
pass_counts['High_pass_Ratio_Total'] = pass_counts['High_pass_Total'] / pass_counts['Total_Passes']
pass_counts['Low_pass_Ratio_Total'] = pass_counts['Low_pass_Total'] / pass_counts['Total_Passes']
pass_counts['Smart_pass_Ratio_Total'] = pass_counts['Smart pass'] / pass_counts['Total_Passes']


# Now, we'll prepare the final DataFrame with the required columns
final_columns = ['Total_Passes', 'High_pass_Ratio_Total', 'Low_pass_Ratio_Total', 'Smart_pass_Ratio_Total']
final_pass_types_df = pass_counts[final_columns]

# If needed, calculate zone-specific ratios (demonstrated for High pass, replicate for Low pass as necessary)
zones = passes_df['Zone'].unique()
for zone in zones:
    zone_specific_passes = passes_df[passes_df['Zone'] == zone]
    zone_pass_counts = zone_specific_passes.groupby(['uniqueTeamId', 'subEventName']).size().unstack(fill_value=0)
    zone_pass_counts['Total_Passes'] = zone_pass_counts.sum(axis=1)
    
    # High pass total and ratio for the zone
    zone_pass_counts['High_pass_Total_Zone'] = zone_pass_counts[high_pass_types].sum(axis=1)
    final_pass_types_df[f'High_pass_Ratio_Zone{zone}'] = zone_pass_counts['High_pass_Total_Zone'] / zone_pass_counts['Total_Passes']
    
    # Low pass total and ratio for the zone
    zone_pass_counts['Low_pass_Total_Zone'] = zone_pass_counts[low_pass_types].sum(axis=1)
    final_pass_types_df[f'Low_pass_Ratio_Zone{zone}'] = zone_pass_counts['Low_pass_Total_Zone'] / zone_pass_counts['Total_Passes']

In [ ]:
final_pass_types_df

# Pass_Connectivity

In [ ]:
pd.DataFrame(passes_df.groupby("matchId"))

In [ ]:
passes_df

In [ ]:
from collections import defaultdict
import networkx as nx

In [ ]:
def calculate_pass_connectivity(group):
    # Initialize two empty graphs for the two teams in the match
    team_graphs = {team_id: nx.Graph() for team_id in group['uniqueTeamId'].unique()}
    
    # Track the previous player and team to identify consecutive passes
    prev_player = None
    prev_team = None
    
    for i, pass_ in group.iterrows():
        current_player = pass_['playerId']
        current_team = pass_['uniqueTeamId']
        
        # Check for consecutive passes within the same team
        if prev_team == current_team and prev_player is not None:
            team_graphs[current_team].add_edge(prev_player, current_player)
        
        # Update the previous player and team for the next iteration
        prev_player = current_player
        prev_team = current_team
    
    # Calculate connectivity for each team graph
    connectivities = {}
    for team_id, graph in team_graphs.items():
        if len(graph) == 0:  # Handle cases where a team made no passes or only incomplete passes
            connectivity = 0
        else:
            connectivity = nx.average_node_connectivity(graph)
        connectivities[team_id] = connectivity
    
    return connectivities

In [ ]:
connectivity_results = [calculate_pass_connectivity(group) for _, group in passes_df.groupby(['matchId'])]

In [ ]:
conn_dict = {**{key: [value] for d in connectivity_results for key, value in d.items()}}
connectivity_df = pd.DataFrame(conn_dict)
connectivity_df = connectivity_df.T
connectivity_df.rename(columns = {0:'Connectivity'}, inplace = True) 

In [ ]:
connectivity_df.to_csv('transition_connectivity_df.csv', index=True)

# Merge DataFrames

In [ ]:
final_df = pd.concat([motif_ratios, final_pass_types_df, pass_direction_df, shot_df_final, connectivity_df], axis=1)
# Iterate through each column in final_df to find and adjust the specified columns
for col in final_df.columns:
    # Check if the column name contains 'Backward', 'Forward', or 'Side'
    if any(keyword in col for keyword in ['Backward', 'Forward', 'Side', "Short", "Middle", "Long"]):
        # Divide the column by the 'Total_Passes' column
        final_df[col] = final_df[col] / final_df['Total_Passes']
        # Rename the column to add " Ratio" at the end
        final_df.rename(columns={col: col + " Ratio"}, inplace=True)

In [59]:
final_df

,ABAB,ABCA,ABCB,ABCD,Total_Passes,High_pass_Ratio_Total,Low_pass_Ratio_Total,Smart_pass_Ratio_Total,High_pass_Ratio_Zone2,Low_pass_Ratio_Zone2,...,Side Pass Ratio,Short Ratio,Middle Ratio,Long Ratio,average_pass_length,Total Shots,Far Shots Ratio,Middle Distance Shots Ratio Ratio,Near Shots Ratio,Connectivity
10531---2499726,0.047619,0.085714,0.085714,0.780952,213,0.145540,0.849765,0.004695,0.060606,0.939394,...,0.474178,0.272300,0.328638,0.399061,25.923929,2.0,0.000000,0.000000,1.000000,5.051282
10531---2499736,0.029915,0.081197,0.081197,0.807692,237,0.151899,0.839662,0.008439,0.067416,0.921348,...,0.497890,0.371308,0.320675,0.308017,25.546512,2.0,0.500000,0.002110,0.000000,5.461538
10531---2499741,0.034043,0.072340,0.072340,0.821277,238,0.168067,0.823529,0.008403,0.153846,0.830769,...,0.478992,0.289916,0.319328,0.390756,26.409845,1.0,0.000000,0.000000,1.000000,5.576923
10531---2499757,0.017921,0.078853,0.075269,0.827957,282,0.156028,0.829787,0.014184,0.095238,0.876190,...,0.478723,0.308511,0.304965,0.386525,26.052560,1.0,0.000000,0.000000,1.000000,5.589744
10531---2499766,0.030675,0.104294,0.110429,0.754601,166,0.234940,0.753012,0.012048,0.144737,0.828947,...,0.379518,0.210843,0.265060,0.524096,30.241780,1.0,0.000000,0.000000,1.000000,4.285714
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
756---2565690,0.044944,0.119850,0.101124,0.734082,270,0.214815,0.777778,0.007407,0.140496,0.851240,...,0.440741,0.296296,0.366667,0.337037,24.859366,5.0,0.400000,0.002222,0.000000,5.230769
756---2565706,0.018634,0.118012,0.111801,0.751553,164,0.213415,0.780488,0.006098,0.118644,0.864407,...,0.445122,0.335366,0.280488,0.384146,28.135415,7.0,0.142857,0.004355,0.142857,4.333333
756---2565714,0.034884,0.046512,0.046512,0.872093,261,0.245211,0.747126,0.007663,0.187500,0.804688,...,0.494253,0.333333,0.314176,0.352490,25.938492,2.0,0.000000,0.001916,0.500000,7.560440
756---2565722,0.024242,0.096970,0.096970,0.781818,168,0.226190,0.767857,0.005952,0.142857,0.833333,...,0.517857,0.285714,0.303571,0.410714,26.801044,4.0,0.000000,0.005952,0.000000,6.064103


In [58]:
table_name = "final_transition_in_poss_df"

if half is not None:
    final_df.to_csv(f'{table_name}_{half}.csv', index=True)
else:
    final_df.to_csv(f'{table_name}.csv', index=True)

# END OF FILE